<a href="https://colab.research.google.com/github/Erofili-K/BI-BA/blob/main/BI_Ask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1η εργασία Επιχειρησιακής Έρευνας & Ευφυΐας

Όνομα: Ερωφίλη Κώνστα

ΑΕΜ: 3618
<br>
<br>

**Άσκηση 1:**
<br>
Πρώτα, κατεβάζω τα δεδομένα (dataset) και τα φορτώνω σε σε ένα Pandas DataFrame. Στην συνέχεια χρησιμοποιώ την describe() για να δω κάποιες μετρικές και το πώς είναι κατανεμημένα τα δεδομένα μέσα στο dataset.
<br>
Η μεταβλητή "ds" περιέχει το dataset της εργασίας.

In [ ]:
import pandas as pd 
import numpy as np

#dataset
!gdown --id 1qklC6HBqynRmhn5q-zY3Ov2a0jcCwT9s

ds = pd.read_csv("/content/GroceriesInitial.csv", delimiter=',', header='infer')
dataf=ds
print(f"This dataset has {ds.shape} rows and collumns")
ds.head(n=10)


In [ ]:
ds.describe()


Το πρώτο πράγμα πριν αρχίσω την επεξεργασία των δεδομένων και προχωρήσω στις ασκήσεις είναι να κάνω οπτικοποίηση των δεδομένων.
<br>
Για αυτό χρησιμοποιώ τις βιβλιοθήκες matplotlib.pyplot & seaborn 
<br>
1. Μια οπτικοποίηση που μπορουμε να κάνουμε είναι ένα ιστόγραμμα που να δείχνει **το πλήθος των συναλλαγών ανά ημέρα**. Για αυτό, πρώτα μετατρέπουμε την στήλη recency_days ,που περιέχει έναν int αριθμό, σε format ημερομηνίας και ώρας.
Επειδή, δεν έχουμε την αρχική ηερομηνία η συνάρτηση pd.to_datetime() χρησιμοποιεί μια αυθαίρετηημερομηνία. Αυτό δεν παίζει ρόλο στην οπτικοποίηση των δεδομένων, γιατί η στήλη recency_days αντιπροσωπεύει το πλήθος ημερών που πέρασαν από τη μέρα της συναλλαγής. Οπότε για παράδειγμα αν σε δυο συναλλαγές το recency_day είναι 5 τότε και οι δυο αυτές συναλλαγές πραγματοποιήθηκαν την ίδια μέρα, άρα στο ιστόγραμμα θα βρίσκονται στην ίδια "ράβδο".
<br> 
Τέλος, μπορούμε να υπλογίσουμε με τον ίδιο τρόπο τον μέσο όρο των συναλλαγών κάθε μέρα.  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

# recency_days -> datetime format
ds['recency_days'] = pd.to_datetime(ds['recency_days'], unit='d')
# ιστογραμμα για ρον αριθμο των συναλλαγων καθε μερα
plt.hist(ds['recency_days'], bins=100)
plt.xlabel('Day')
plt.ylabel('Number of transactions')
plt.show()
# μο συναλλαγες ανά ημέρα
daily_transactions = ds.groupby(ds['recency_days'].dt.date)['basket_value'].mean()
print(f"Mean number of transactions each day: {daily_transactions.mean()} or rounded up: {math.ceil(daily_transactions.mean())}")

2. Μια δεύτερη αρχική οπτικοποίηση των δεδομένων που μπορεί να γίνει με το dataset είναι να δημιουργήσουμε ένα ιστογραμμα που δείχνει πόσα προϊόντα αγοράζονται συνήθως ανά συναλλαγή. Αρχικά μετράμε πόσες στήλες απο την item_1 - item_32 έχουν τιμή και δεν είναι κενές και φτιάχνουμε το ιστόγραμμα. 
<br>
Παρατηρούμε ότι, οι περισσότεροι καταναλωτές αγοράζουν συνήθως από 1-6 προιόντα.

In [ ]:
# πληθος προιοντων ανα συνναλλαγη
c=[]
for i, row in ds.iterrows():
    count = row.iloc[4:].count()
    #print(f"id: {row['id']} {count} items bought")
    c.append(count)

plt.hist(c, bins=100)
plt.xlabel("Number of items bought")
plt.ylabel("Frequency")
plt.show()

Στην συνέχεια πρέπει να κάνουμε μετασχηματισμό των εδομένων σε δυαδική μορφή συναλλαγών.
<br> 
Πρώτα, δημιουργούμε μια λίστα lst  που περιέχει όλα τα ονόματα των προιόντων που αγοράζουν οι καταναλωτές, χωρίς τις nan τιμές και χωρίς διπλότυπα, από τις τιμές που βρίσκονται στις στήλες item_1-item32. Στην συνέχεια ταξινομούμε την λίστα ώστε να έχει αλφαβητική σειρά και τέλος μπορούμε να εμφάνισουμε τον αριθμό των στοιχείων που περιέχει για να ελέγξουμε ότι όλα τα προιόντα έχουν περασθεί στην λίστα.
<br> 

In [ ]:
import numpy as np
#κραταμε μονο τις τιμες απο την 3η στηλη κ μετα
lst = ds.iloc[:, 3:].values.flatten().tolist()
#print(lst)
#σβηνουμε τα nan
lst = [x for x in lst if not pd.isna(x)]
#μοναδικα στοιχια(οχι διπλοτυπα)
lst = list(set(lst))
print(lst)
#print(len(lst))
#sort A->z
lst.sort()
print(lst)
print(len(lst))

Μετά, δημιουργούμε τον δυαδικό πίνακα σε dataframe όπου κάθε στήλη αποτελεί ένα προιόν και κάθε γραμμή μια συναλλαγή. Σε κάθε κελί τοποθετείται 0 αν το προιόν δεν έχει αγορασθεί και 1 αν έχει αγορασθεί.
Τέλος, κρατάμε μόνο τα προιόντα που μας ενιδαφέρουν τα οποία είναι: citrus fruit, tropical fruit, whole milk, other vegetables, rolls/buns, chocolate, bottled water, yogurt,
sausage, root vegetables, pastry, sodα, cream. Για κάθε συναλλαγή, βρίσκουμε τα προϊόντα που αγοράστηκαν και για όσα δεν έχουν αγορασθεί βάζουμε 0.


In [ ]:
# κενο dataframe με τον αριθμό των γραμμών του αρχικού dataframe και τον αριθμό των στηλών της λίστας(=169)
new_df = pd.DataFrame(0, index=ds.index, columns=lst)
# Ορισμός των ονομάτων των στηλών του νέου dataframe ως τα ονόματα των προϊόντων στη λίστα
new_df.columns = lst
products = list(set(lst))
products.sort()
# create binary matrix
new_df = pd.DataFrame(columns=products)
for i, transaction in enumerate(ds.values):
    row = [0] * len(products)
    for product in products:
        if product in transaction:
            row[products.index(product)] = 1
    new_df.loc[i] = row
#print(new_df.head())
selected_p = ['citrus fruit', 'tropical fruit', 'whole milk', 'other vegetables', 'rolls/buns', 'chocolate', 'bottled water', 'yogurt', 'sausage', 'root vegetables', 'pastry', 'soda', 'cream']
# Δημιουργία νέου dataframe
new_df = pd.DataFrame(columns=lst)
# Για κάθε συναλλαγή, βρίσκουμε τα προϊόντα που αγοράστηκαν
for i in range(len(ds)):
    products = ds.iloc[i, 3:].tolist()
    products = [p for p in products if p in selected_p]
    for product in products:
        new_df.at[i, product] = 1
# Αντικατάσταση των NaN με 0
new_df = new_df.fillna(0)
old_df = new_df
new_df = new_df[selected_p]
new_df.head(n=20)


Στο παρακάτω ιστόγραμμα μπορούμε να δούμε την κατανομή των προιόντων που μας ενδιαφέρουν.

In [ ]:
# Υπολογίζουμε τον αριθμό των αγορών που περιλαμβάνουν το κάθε προϊόν
counts = new_df.sum().sort_values(ascending=False)[:169]

# Δημιουργούμε το διάγραμμα αναλογιών
fig, ax = plt.subplots(figsize=(8,6))
ax.bar(counts.index, counts.values, color='blue')
ax.set_title('Most Frequently Bought Items')
ax.set_xlabel('Item')
ax.set_ylabel('Number of Transactions')
plt.xticks(rotation=45)
plt.show()

Τέλος, κάνουμε την διακριτοποίηση των τιμών της στήλης basket_value και οπτικοποιούμε τα αποτελέσματα σε δυο μορφές: ιστόγραμμα και πίτα.
Αρχικά, εμφανίζουμε πόσες τιμές περιέχει κάιε στήλη για να ελέγξουμε ότι ειναι περίπου ισοπληθείς. Μετά δημιουργούμε πρώτα ένα bar chart και στην συνέχεια ένα pie chart για την οπτικοποίηση.

In [ ]:
# Διακριτοποίηση της στήλης "basket_value" σε τρεις ισοπληθείς κατηγορίες
basket_value_bins = pd.qcut(ds['basket_value'], q=3, labels=['low_value_basket', 'medium_value_basket', 'high_value_basket'])
ds['basket_category'] = basket_value_bins

# Υπολογίζουμε τον αριθμό των αγορών που ανήκουν σε κάθε κατηγορία
counts = ds['basket_category'].value_counts()
print(counts)
# Δημιουργούμε το ιστόγραμμα
fig, ax = plt.subplots()
ax.bar(counts.index, counts.values, color='blue')
ax.set_title('Number of Transactions by Basket Category')
ax.set_xlabel('Basket Category')
ax.set_ylabel('Number of Transactions')
plt.show()

In [ ]:
# Οπτικοποίηση ως pie chart
fig, ax = plt.subplots(figsize=(6,6))
ax.pie(counts.values, labels=counts.index, autopct='%1.1f%%', startangle=90)
ax.set_title('Distribution of Basket Categories')
plt.show()

**Άσκηση 2**
<br>
Μάθηση κανόνων συσχέτισης με την μέθοδο Apriori:
<br>
i.  Δοκιμάστε την εκτέλεση της μεθόδου Αpriori με διάφορες παραμέτρους για το ελάχιστο
Support
<br>
Πρώτα κάνουμε install το package apyori της python και τις βιβλιοθήκες που θα μας χρειαστούν. Στην συνέχεια φτιάχνουμε μια λίστα που περιέχει τον δυαδικό πίνακα που δημιουργησαμε στοην προηγουμενη ασκηση για να περασθεί ως όρισμα στην συνάρτηση apriori για να βγάλουμε τουςκανόνες.



In [ ]:
!pip install apyori

In [ ]:
from pandas.core.internals.managers import new_block
from apyori import apriori

#σειρες και στηλες
print(new_df.shape)
r=new_df.shape[0]
c=new_df.shape[1]

# Intializing the list
transacts = []
#  a list of transactions
for i in range(0, r): 
  transacts.append([str(new_df.values[i,j]) for j in range(0, c)])
#print(transacts)
# Convert data to list format and run Apriori algorithm
transactions = list(transacts)
print(transactions)
rules = apriori(transactions, min_support=0.03, min_confidence=0.02)
list(rules)




In [ ]:

# putting output into a pandas dataframe
def inspect(output):
    lhs         = [tuple(result[2][0][0])[0] for result in output]
    rhs         = [tuple(result[2][0][1])[0] for result in output]
    support    = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift       = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))
output = inspect(rules)
output_DataFrame = pd.DataFrame(output, columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
#output_DataFrame.head(n=20)
print(output_DataFrame)



# **Ασκηση 3**
Αρχικά, δημιουργουμε ενα dataframe που περιέχει τις 2 πρωτες στήλες που μας ενδιαφέρουν , δηλαδη 'basket_value', 'recency_days'.
Στο πρώτο κελί κώδικα έχουμε την εντολή dataf=ds ώστε να σημιουργηθεί ένα copy του dataset για να μπορέσουμε να "κόψουμε" απο το αρχικό dataset τις 2 στηλες που μας ενδιαφερουν και να κάνουμε αλλαγές χωρίς να πειράξουμε το αρικό. 
<br> 
Για να βεβαιωθουμε οτι τρεχει σωστά ο παρακάτω κώδικας, πρέπει μόλις εμφανιστεί ο πίνακς στην στηλη recencydays να φαίνονται οι ακαιρεοι αριθμοί και όχι ημερομηνίες. Αν φαίνονται ημερομηνίες απλά τρέχουμε ξανά το πρωτο block κώδικα (εκεί που γίνεται το copy του dataset). Και τέλος για να τρέξουμε kmeans χρησιμοπιούμε την βιβλιοθήκη sklearn

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

selected_cols = ['basket_value', 'recency_days']
new_d = dataf[selected_cols]
new_d.head(n=10)

Στην συνέχεια, σε μια πρωτη αναλαυση, κανουμε κανονικοποιηση στα δεδομένα του παραπάνω πίνακα (new_d) και χρησιμοποιουμε την συναρτηση KMeans() με αριθμό clusters=5 ,συμφωνα με την εκφωνηση. Και τέλος παρουσιάζονται τε κεντρα των 5 ομάδω και τα αποτελεσματα σε ενα γραφημα.
<br> 
Στο γραφημα παρατηρουμε ότι, ενώ καποια clusters έχουν διαχωρισθεί "σωστα" υπάρχουν κάποια σημεία όπου μάλλον ανήκουν σε άλλη ομάδα από αυτην που έχουν εισαχθεί.

In [ ]:
# κανονικοποίηση των δεδομένων
scaler = StandardScaler()
scaled_data = scaler.fit_transform(new_d)

# εφαρμογή του αλγορίθμου k-means
kmeans = KMeans(n_clusters=5)
kmeans.fit(scaled_data)

# εκτύπωση των κέντρων των κλάσεων
print(kmeans.cluster_centers_)

# παρουσίαση των αποτελεσμάτων σε γράφημα
plt.scatter(scaled_data[:, 0], scaled_data[:, 1], c=kmeans.labels_, cmap='rainbow')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color='black')
plt.title('Clustering results')
plt.xlabel('Basket value')
plt.ylabel('Recency days')
plt.show()








Στην συνέχεια, παιρνουμε τα δεδομένα και αφου βγάλουμε τους 5 clusters , κάνουμε τους υπολογισμους των μέσων τιμών και των τυπικών αποκλίσεων των κέντρων των ομάδων
<br>
Στον παρακάτω κωδικα:
<br>
Επιλέγουμε τον αριθμό των ομάδων που θέλουμε να δημιουργήσουμε και δημιουργούμε ένα αντικείμενο KMeans με αυτόν τον αριθμό ως παράμετρο.

Περνάμε τα δεδομένα μας στον clusterer χρησιμοποιώντας τη μέθοδο fit().

Χρησιμοποιούμε τη μέθοδο predict() για να αντιστοιχίσουμε κάθε παρατήρηση στην ομάδα που ανήκει.
<br>
Στη συνέχεια, μπορούμε να ερμηνεύσουμε κάθε ομάδα μέσω των μέσων τιμών και των τυπικών αποκλίσεων των κέντρων των ομάδων, σε συνδυασμό με την κατανομή των συναλλαγών που ανήκουν σε κάθε ομάδα. Για παράδειγμα, μπορούμε να έχουμε την εξής ερμηνεία των ομάδων:
<br>
 Η Ομάδα 1 και 2 αντιπροσωπεύουν συναλλαγές που έχουν γίνει πρόσφατα, με διαφορετικές αξίες, ενώ η Ομάδα 3 και 4 αντιπροσωπεύουν συναλλαγές με διαφορετικές αξίες που έχουν γίνει παλαιότερα, και η Ομάδα 5 αντιπροσωπεύει συναλλαγές υψηλής αξίας που έχουν γίνει παλαιότερα.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X = new_d.values
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=20, alpha=1);

#τυπ αποκλ & μεση τιμη
centers = kmeans.cluster_centers_
mean_values = centers.mean(axis=0)
std_values = centers.std(axis=0)
print(f'centers: {centers} \n mean values: {mean_values} \n std values: {std_values}')
for i in range(len(centers)):
    print("Cluster {}: Mean - {}, Standard Deviation - {}".format(i+1, centers[i], X[y_kmeans == i].std(axis=0)))

Για το 3ο ερωτημα της ασκησης 3, δημιουργουμε ένα dataframe που περιεέχει τις στηλες basket_value & recency_days και ως έξτρα έχουμε την κατηγορια στην οποια ανηκει καθε συναλλαγη (0-4 αναλογα σε ποιο cluster ειναι) και μια στηλη για καθε cluster οπου εχει 0/1 αναλογα με το αν η συναλλαγη ανηκει σε αυτον η οχι.

In [ ]:
# Ανάθεση συναλλαγών σε cluster
y_kmeans = kmeans.predict(X)

# Δημιουργία νέου χαρακτηριστικού
cluster_cols = pd.get_dummies(y_kmeans, prefix='Cluster')

# Συνένωση των δύο πίνακων
new_d = pd.concat([new_d, cluster_cols], axis=1)

# Εμφάνιση των πρώτων 10 συναλλαγών με τα νέα χαρακτηριστικά
print(new_d.head(10))



Εκπαίδευση του μοντέλου σε συνθετικά δεδομένα.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
for n_clusters in range(2,10):
      # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()